## Importing Necessary Libaray

In [2]:
import numpy as np
import pandas as pd
import shap
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from pymoo.algorithms.moo.nsga3 import NSGA3
from pymoo.core.problem import Problem
from pymoo.optimize import minimize
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.operators.crossover.sbx import SimulatedBinaryCrossover
from pymoo.operators.mutation.pm import PolynomialMutation
from sklearn.metrics import accuracy_score, log_loss, confusion_matrix, f1_score, roc_auc_score, roc_curve, auc, precision_score, recall_score
from pymoo.core.problem import ElementwiseProblem
from pymoo.util.normalization import normalize
import matplotlib.pyplot as plt
import seaborn as sns
from pymoo.util.ref_dirs import get_reference_directions
from pymoo.core.callback import Callback

## Loading Haberman Dataset

In [5]:
scaler = StandardScaler()   ## Standarizing Function
df_Hab = pd.read_csv("Haberman.csv") ## Reading Haberman Dataset
data = df_Hab

df_Hab.replace('?', np.nan, inplace=True) ## Replace '?' with NaN
df_Hab = df_Hab.apply(pd.to_numeric, errors='coerce') ## Convert columns to numeric, forcing errors to NaN
means = df_Hab.mean() ## Calculate mean of each column (ignoring NaNs)
df_Hab.fillna(means, inplace=True) ## Replacing NaN values with the corresponding mean

## Extracting Features and Labels

In [6]:
X_Hab = df_Hab.iloc[: , 0:3].values
X_Hab = scaler.fit_transform(X_Hab)  ## Standaarizing the data

y_Hab = data['Label'].map({1:0, 2:1}).values
Num_Samples , Num_Features = X_Hab.shape
Num_Classes = df_Hab['Label'].nunique()
print(Num_Samples , Num_Features , y_Hab.shape)

306 3 (306,)


## Segregating Data into Training and Testing Data

In [7]:
X_train_Hab , X_test_Hab , y_train_Hab , y_test_Hab = train_test_split(X_Hab , y_Hab , test_size = 0.3 , random_state = 42)

print(f"Train dataset size: {X_train_Hab.shape} , Train dataset label size: {y_train_Hab.shape}")
print(f"Test dataset size: {X_test_Hab.shape} , Test dataset label size: {y_test_Hab.shape}")

Train dataset size: (214, 3) , Train dataset label size: (214,)
Test dataset size: (92, 3) , Test dataset label size: (92,)


## Defining Logistic Function

In [8]:
class LogisticFunctionModule:
    def __init__(self, n_features):
        self.weights = np.zeros(n_features + 1)  ## Initializing weights and bias term

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def predict(self, X):
        z = np.dot(X, self.weights[1:]) + self.weights[0]
        return self.sigmoid(z)

    def compute_loss(self, X, y):
        m = len(y)
        predictions = self.predict(X)
        loss = - (1/m) * (np.dot(y, np.log(predictions + 1e-15)) + np.dot(1 - y, np.log(1 - predictions + 1e-15)))
        return loss

## Defining Many-Objective Function Including Model's Loss and Feature Importance of All Features

In [9]:
class MultiObjectiveExplainableProblem(ElementwiseProblem):
    def __init__(self , X_train , y_train):
        self.X_train = X_train
        self.y_train = y_train
        super().__init__(n_var = X_train.shape[1] + 1,  ## Number of learnable varaibales (weights and bias parameters)
                         n_obj = X_train.shape[1] + 1,  
                         n_constr = 0,
                         xl = -1,  
                         xu = 1)   

    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def predict(self, X):
        z = np.dot(X, self.weights[1:]) + self.weights[0]
        return self.sigmoid(z)

    def compute_loss(self, X, y):
        m = len(y)
        predictions = self.predict(X)
        loss = - (1/m) * (np.dot(y, np.log(predictions + 1e-15)) + np.dot(1 - y, np.log(1 - predictions + 1e-15)))
        return loss

    def compute_metrics(self , X , y):
        predictions = self.predict(X) >= 0.5
        accuracy = accuracy_score(y , predictions)
        precision = precision_score(y , predictions)
        recall = recall_score(y , predictions)
        f1_val = f1_score(y , predictions)
        tn , fp , fn , tp = confusion_matrix(y , predictions).ravel()
        specificity = tn / (tn+fp)
        return accuracy , precision , recall , specificity , f1_val
        

    def _evaluate(self, x, out, *args, **kwargs):
        
        self.weights = x
        
        loss = self.compute_loss(self.X_train , self.y_train)

        explainer = shap.Explainer(self.predict, self.X_train)
        shap_values = explainer(self.X_train)

        mean_shap_values = np.mean(shap_values.values , axis = 0 ) 

        objectives = np.array([loss] + [-v for v in mean_shap_values])
        
        out["F"] = objectives
        

## Implementing MoEC Algorithm

In [12]:
Num_Obj = X_train_Hab.shape[1] + 1
Ref_Dirs = get_reference_directions("energy", Num_Obj, n_points=100)
print(f"Number of reference directions: {len(Ref_Dirs)}") ## Number of reference directions generated


algorithm = NSGA3(
    pop_size = len(Ref_Dirs),  # Set population size equal to reference directions
    ref_dirs = Ref_Dirs,
    n_offsprings = 30,
    sampling = FloatRandomSampling(),
    crossover = SimulatedBinaryCrossover(prob = 0.9 , eta = 15),
    mutation = PolynomialMutation(eta = 20),
    eliminate_duplicates = True
)

problem = MultiObjectiveExplainableProblem(X_train_Hab, y_train_Hab)

res = minimize(problem,
               algorithm,
               termination = ('n_gen', 10),
               seed = 1,
               verbose = True,
              )

Number of reference directions: 100
n_gen  |  n_eval  | n_nds  |      eps      |   indicator  
     1 |      100 |     26 |             - |             -
     2 |      130 |     33 |  0.0164984633 |         ideal
     3 |      160 |     31 |  0.0135589852 |         ideal
     4 |      190 |     33 |  0.0154017013 |         nadir
     5 |      220 |     36 |  0.0117273373 |         ideal
     6 |      250 |     38 |  0.0180907823 |         ideal
     7 |      280 |     44 |  0.3862799832 |         nadir
     8 |      310 |     45 |  0.0421894889 |         ideal
     9 |      340 |     41 |  0.1157821693 |         nadir
    10 |      370 |     40 |  0.0178322881 |             f


## Computing Quantitaive Metrics for Test Data

In [ ]:
## Extracting the best solution
for i in range(len(res.F)):
    
    best_solution = res.X[i]
    
    model = LogisticFunctionModule(n_features = X_train_Hab.shape[1])
    model.weights = best_solution 
    
    def evaluate_model(model, X, y):
        probabilities = model.predict(X)
        predictions = model.predict(X) >= 0.5
        accuracy = np.mean(predictions == y)
        sensitivity = np.sum((predictions == 1) & (y == 1)) / np.sum(y == 1)
        specificity = np.sum((predictions == 0) & (y == 0)) / np.sum(y == 0)
        f1_val = 2 * (sensitivity * np.sum((predictions == 1) & (y == 1)) / np.sum(predictions == 1)) / (sensitivity + np.sum((predictions == 1) & (y == 1)) / np.sum(predictions == 1))
        precision = precision_score(y , predictions)
        auc = roc_auc_score(y, probabilities)
        return accuracy, sensitivity, specificity, f1_val, precision, auc
    
    # Evaluate on test set
    accuracy, sensitivity, specificity, f1_score, precision, auc = evaluate_model(model, X_test_Hab, y_test_Hab)
    print(f"Soluion for :{i}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Test Sensitivity: {sensitivity:.4f}")
    print(f"Test Specificity: {specificity:.4f}")
    print(f"Test F1 Score: {f1_score:.4f}")
    print(f"Test Precision Score: {precision:.4f}")
    print(f"Test AUC Score: {auc:.4f}")
    print("-----------------------------------")
